# Some example demographies

We illustrate some advanced options of `momi.make_demography()`.

In [1]:
from momi import make_demography, to_ms_cmd

### Specifying demographies in different time scales

Users can choose to input demographies in ms time scale, the more 'standard' coalescent time scale, or in per-generation units.

In [2]:
# a simple Demography, in ms units
events = [('-eg', 0.0, 'a', 1.0), ('-ej', .5, 'a', 'b'), ('-en', 1., 'b', .5)]
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3))

print "Demography in ms syntax:"
print to_ms_cmd(demo)

Demography in ms syntax:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [3]:
# the same Demography, but specified in the 'standard' (not 'ms') time scale
# this means that there are 2*N_ref generations per unit time (instead of 4*N_ref as in ms)

# notice times are 2x longer in the 'standard' units.
# as consequence, growth rate per unit time is 2x slower
events = [('-eg', 0.0, 'a', 0.5), ('-ej', 1., 'a', 'b'), ('-en', 2., 'b', .5)]

# use time_scale='standard' to specify standard coalescent units
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3), time_scale='standard')

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [4]:
# the same Demography, but specified in per-generation units
# we'll assume a diploid population size of 10,000

# join event 5e3 generations ago. population size drops to 5000 at 1e4 generations ago.
# growth rate per generation is 1e-4 in population 'a'
events = [('-eg',0.0,'a',1e-4),('-ej', 5e3, 'a', 'b'), ('-en', 1e4, 'b', 5000)]

# use default_N to set the diploid population size
demo = make_demography(events, sampled_pops=('a','b'), sampled_n=(5,3), default_N=10000)

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


### Ghost populations

A 'ghost' population is one that is not present in the sample.

`momi` syntax is more flexible than `ms` about ghost populations. In particular, ghost populations can be immediately affected by demographic events.

By contrast, in `ms`, ghost populations cannot be modified by events until they have been "created" with the **-es** flag.

In [5]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0), # can set growth rate of 'z' at t=0, even tho its a ghost population
          ('-ep', .5, 'a', 'z', .25), # pulse migration from z to a
          ('-ej', 1.0, 'z', 'a')]

demo = make_demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the growth event (-eg) doesn't happen until the admixture event (-es)

Demography in ms syntax.
-I 1 10 -es 0.5 1 0.75 -en 0.5 2 0.606530659713 -eg 0.5 2 1.0 -ej 1.0 2 1


In `momi`, we can also have sampled populations merge **-ej** with ghost populations, even if there is no admixture event.

In [6]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0),
          ('-ej', .5, 'a', 'z'), # 'z' is empty, so this is equivalent to just changing size/growth of 'a'
          ('-en', 1.0, 'z', 1.0)]

demo = make_demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the -ej event doesn't appear; instead just the size and growth rate of 'a' change

Demography in ms syntax.
-I 1 10 -en 0.5 1 0.606530659713 -eg 0.5 1 1.0 -en 1.0 1 1.0


### Ancient DNA

Use the `sampled_t` parameter to indicate that a population was sampled in the past.

In [7]:
events = [('-ej', 1.0, 'a', 'b')]

# the samples from 'b' are ancient DNA, from time t=0.5
demo = make_demography(events, ['a','b'], (6, 10), sampled_t=[0.0, .5])

try:
    print to_ms_cmd(demo)
except:
    print "Cannot convert demography with archaic populations to ms syntax"

Cannot convert demography with archaic populations to ms syntax


### Demography with a lot of parameters

An example on how to programatically construct a demography with a lot of parameters (too many to conveniently write down by hand).

In [8]:
# piecewise constant demography, with 50 pieces

import random

n_pieces = 50
t = 0.0
events = []
for i in range(n_pieces):
    t = t + random.expovariate(100.)
    N = random.expovariate(1.0)
    events.append(('-en', t, 0, N))

demo = make_demography(events, [0], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)

Demography in ms syntax.
-I 1 10 -en 0.00708765506784 1 1.62015594578 -en 0.0313323909819 1 0.315924810311 -en 0.0397956713291 1 1.33932598479 -en 0.06736000592 1 0.765217226833 -en 0.0811446703531 1 0.87155433285 -en 0.0914700248963 1 3.04704161042 -en 0.0963927307666 1 0.383427246569 -en 0.0964724602381 1 1.85933331288 -en 0.101392361799 1 3.08489832534 -en 0.124937509912 1 0.304676696588 -en 0.154937289801 1 1.43589587026 -en 0.17103974982 1 1.71644503485 -en 0.171749141991 1 2.33907421362 -en 0.173706316695 1 0.956785724005 -en 0.17906645711 1 0.194663038951 -en 0.180153262974 1 0.365729333027 -en 0.187268934459 1 0.126274191666 -en 0.190916024916 1 0.054391990582 -en 0.194319934962 1 0.165624897869 -en 0.198253950568 1 0.350463183469 -en 0.219017236604 1 0.222393057677 -en 0.22204353809 1 0.392171991758 -en 0.223517093964 1 0.5307132388 -en 0.239053611396 1 0.125025574399 -en 0.243283880291 1 0.100238390715 -en 0.267737045183 1 0.26435995293 -en 0.280024373064 1 0.759256420681 -en